In [3]:
import chainer
import os

## Model

In [2]:
# This network builds a bit different network that add layers that we need to be able to learn our translation
# First we need the translation layer 
# Secondly we need to the predict the next embedding instead of the word itself. If not we cannot re-use the network for the other language
# We can however also no train the word embedding immid
class TransformingRNNLM(chainer.Chain):

    def __init__(self, lm, train=True):
        super(TransformingRNNLM, self).__init__(
            embed=L.EmbedID(n_vocab, n_units),
            l0=L.Linear(n_units,n_units,nobias=True),
            l1=L.LSTM(n_units, n_units),
            l2=L.LSTM(n_units, n_units),
            l3=L.Linear(n_units, n_vocab),
        )
        
        # Initialize with uniform distribution, expect for our linear tranformation layer
        for param in self.params():
            param.data[...] = np.random.uniform(-0.1, 0.1, param.data.shape)
        
        # Our linear tranformation layer starts with 
        for param in self.l0.params():
            param.data[...] = np.eye(n_units)
        self.train = train

    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def __call__(self, x):
        h0 = self.embed(x)
        h1 = self.l0(h0)
        h2 = self.l1(F.dropout(h2, train=self.train))
        h3 = self.l2(F.dropout(h3, train=self.train))
        y = self.l3(F.dropout(h2, train=self.train))
        return y